# Week 12: Agents & Orchestration

Building an autonomous agent with ReAct pattern.

## Learning Objectives
1. Implement Tool Abstractions
2. Build the ReAct (Reasoning + Acting) Loop
3. Create a Simple Agent

In [ ]:
import re
from typing import List, Callable

## 1. Tools Definition

In [ ]:
class Tool:
    def __init__(self, name: str, func: Callable, description: str):
        self.name = name
        self.func = func
        self.description = description
    
    def run(self, input_str: str) -> str:
        return str(self.func(input_str))

# Sample tools
def calculator(expression: str):
    try:
        return eval(expression)
    except:
        return "Error"

def search(query: str):
    return f"Search results for: {query} (Simulated)"

tools = [
    Tool("CALCULATOR", calculator, "Useful for math. Input: number expression"),
    Tool("SEARCH", search, "Useful for info. Input: query string")
]

## 2. ReAct Agent

Pattern: Thought -> Action -> Observation -> Thought...

In [ ]:
class Agent:
    def __init__(self, tools: List[Tool]):
        self.tools = {t.name: t for t in tools}
        
    def prompt_template(self, query: str):
        tool_desc = "\n".join([f"{t.name}: {t.description}" for t in self.tools.values()])
        return f"""Answer the following question as best you can.
You have access to the following tools:
{tool_desc}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{', '.join(self.tools.keys())}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {query}
Thought:"""
    
    def mock_llm_response(self, prompt: str):
        # Simulating LLM logic for demo
        if "Final Answer" in prompt: return "DONE"
        last_line = prompt.strip().split('\n')[-1]
        
        if "Question: What is 2 + 2" in prompt:
            if "Observation" not in prompt:
                return "I need to calculate this. \nAction: CALCULATOR\nAction Input: 2 + 2"
            else:
                return "I have the result. \nFinal Answer: 4"
        return "Final Answer: I don't know"

    def run(self, query: str):
        prompt = self.prompt_template(query)
        print(prompt)
        
        for _ in range(5):
            response = self.mock_llm_response(prompt)
            print(f"> {response}")
            
            if "Final Answer:" in response:
                return response.split("Final Answer:")[1].strip()
            
            # Parse Action
            match = re.search(r"Action: (\w+)\nAction Input: (.+)", response, re.DOTALL)
            if match:
                action = match.group(1)
                action_input = match.group(2)
                
                if action in self.tools:
                    observation = self.tools[action].run(action_input)
                    print(f"Observation: {observation}")
                    prompt += f"{response}\nObservation: {observation}\nThought:"
                else:
                    print("Error: Unknown tool")
                    break
            else:
                break

In [ ]:
# Test Agent
agent = Agent(tools)
agent.run("What is 2 + 2")